# 3-5 資料分析師的技能供給

> 補充教材

郭耀仁

## 關於布林篩選

常見資料處理任務中，我們常會要進行類似 SQL 基礎查詢的操作：又稱為資料控制文法（The Grammar of Data Manipulation），包含選擇、篩選、衍生、排序、摘要與分組六種可以和 SQL 基礎查詢完美呼應的技巧，在 Python 中可以使用套件 pandas 來實踐。要從資料框中挑出職稱為 Data Analyst 的受訪者資料，我們可以應用**篩選**這個技法，並且使用布林（`boolean`）運算符：`==`，亦即**等於**。

In [1]:
import pandas as pd

In [2]:
survey_2017 = pd.read_csv("https://kaggle-ml-ds-survey.s3-ap-northeast-1.amazonaws.com/kaggle-survey-2017/multipleChoiceResponses.csv", encoding='ISO-8859-1', low_memory=False)
survey_2018 = pd.read_csv("https://kaggle-ml-ds-survey.s3-ap-northeast-1.amazonaws.com/kaggle-survey-2018/multipleChoiceResponses.csv", encoding='ISO-8859-1', skiprows=[1], low_memory=False)
survey_2019 = pd.read_csv("https://kaggle-ml-ds-survey.s3-ap-northeast-1.amazonaws.com/kaggle-survey-2019/multiple_choice_responses.csv", encoding='ISO-8859-1', skiprows=[1], low_memory=False)

In [3]:
# 使用布林（boolean）運算符：==
survey_2017['CurrentJobTitleSelect'] == 'Data Analyst'

0        False
1        False
2        False
3        False
4        False
         ...  
16711    False
16712    False
16713    False
16714    False
16715    False
Name: CurrentJobTitleSelect, Length: 16716, dtype: bool

In [4]:
# 使用布林（boolean）運算符：==
survey_2018['Q6'] == 'Data Analyst'

0        False
1        False
2        False
3        False
4         True
         ...  
23854    False
23855    False
23856    False
23857    False
23858    False
Name: Q6, Length: 23859, dtype: bool

In [5]:
# 使用布林（boolean）運算符：==
survey_2019['Q5'] == 'Data Analyst'

0        False
1        False
2        False
3        False
4        False
         ...  
19712    False
19713    False
19714    False
19715    False
19716    False
Name: Q5, Length: 19717, dtype: bool

In [6]:
# 篩選：保留布林條件判斷為 True 的觀測值
da_2017 = survey_2017[survey_2017['CurrentJobTitleSelect'] == 'Data Analyst']
da_2018 = survey_2018[survey_2018['Q6'] == 'Data Analyst']
da_2019 = survey_2019[survey_2019['Q5'] == 'Data Analyst']

## 關於自訂函式 `get_value_ratios`

我們希望可以將 `da_2017`、`da_2018` 與 `da_2019` 這三個資料框中單選題所記錄的比例回傳，但是不希望每更動一次資料框、每更動一個單選題，就要重新撰寫一次大抵相同的程式碼，在這裡就可以運用**自訂函式**的技巧，宣告一個名稱為 `get_value_ratios` 函式，設計兩個參數：`df` 與 `col_name`，分別代表資料框名稱與單選題的題目名（或題號）。

In [7]:
def get_value_ratios(df, col_name):
    return df[col_name].value_counts() / df[col_name].value_counts().sum()

In [8]:
# 以 da_2017['JobFunctionSelect'] 為例
# .value_counts() 方法能回傳所有受訪者在某個單選題的答題狀況
da_2017['JobFunctionSelect'].value_counts()

Analyze and understand data to influence product or business decisions                                                631
Build and/or run the data infrastructure that your business uses for storing, analyzing, and operationalizing data     98
Build and/or run a machine learning service that operationally improves your product or workflows                      73
Build prototypes to explore applying machine learning to new areas                                                     47
Other                                                                                                                  27
Research that advances the state of the art of machine learning                                                        11
Name: JobFunctionSelect, dtype: int64

In [9]:
# 在 .value_counts() 方法後接著呼叫 .sum() 方法則是將所有的答題人數加總起來
da_2017['JobFunctionSelect'].value_counts().sum()

887

In [10]:
# 兩者相除就可以得到單選題的答覆比例
da_2017['JobFunctionSelect'].value_counts() / da_2017['JobFunctionSelect'].value_counts().sum()

Analyze and understand data to influence product or business decisions                                                0.711387
Build and/or run the data infrastructure that your business uses for storing, analyzing, and operationalizing data    0.110485
Build and/or run a machine learning service that operationally improves your product or workflows                     0.082300
Build prototypes to explore applying machine learning to new areas                                                    0.052988
Other                                                                                                                 0.030440
Research that advances the state of the art of machine learning                                                       0.012401
Name: JobFunctionSelect, dtype: float64

In [11]:
# 不想重複做前述的事情，直接呼叫 get_value_ratios 函式
get_value_ratios(da_2017, 'JobFunctionSelect')

Analyze and understand data to influence product or business decisions                                                0.711387
Build and/or run the data infrastructure that your business uses for storing, analyzing, and operationalizing data    0.110485
Build and/or run a machine learning service that operationally improves your product or workflows                     0.082300
Build prototypes to explore applying machine learning to new areas                                                    0.052988
Other                                                                                                                 0.030440
Research that advances the state of the art of machine learning                                                       0.012401
Name: JobFunctionSelect, dtype: float64

## 關於自訂函式 `get_checkbox_ratios`

我們希望可以將 `da_2017`、`da_2018` 與 `da_2019` 這三個資料框中複選題所記錄的比例回傳，但是不希望每更動一次資料框、每更動一個單選題，就要重新撰寫一次大抵相同的程式碼，在這裡就可以運用**自訂函式**的技巧，宣告一個名稱為 `get_checkbox_ratios` 函式，設計兩個參數：`df` 與 `col_pattern`，分別代表資料框名稱與複選題的題目特徵（或題號特徵）。

In [12]:
def get_checkbox_ratios(df, col_pattern):
    # 初始化兩個空的 list 用來存放複選題的選項與勾選數
    channel_names = [] # 複選題的選項
    channel_checks = [] # 複選題的勾選數
    for col_name in df.columns:
        # 將符合複選題的題目特徵（或題號特徵）的變數欄位納入
        # 排除題目特徵中有 OTHER_TEXT 這種開放式回答的變數欄位
        if (col_pattern in col_name) and ('OTHER_TEXT' not in col_name):
            channel_name = df[col_name].value_counts().index # 取出複選題選項
            channel_check = df[col_name].value_counts().values # 取出複選題的勾選數
            # 排除空字串的複選題選項
            if channel_name.size != 0:
                channel_names.append(channel_name[0]) # 存入複選題選項
                channel_checks.append(channel_check[0]) # 存入複選題勾選數
    channel_counts = pd.Series(channel_checks, index=channel_names) # 製作一個好排序的 pandas Series
    channel_ratios = channel_counts / channel_counts.sum() # 加總
    channel_ratios_sorted = channel_ratios.sort_values(ascending=False) # 遞減（由大到小）排序複選題的勾選數
    return channel_ratios_sorted

In [13]:
# 以 da_2018['Q11_XXX'] 為例
da_2018['Q11_Part_1']

4                                                      NaN
14       Analyze and understand data to influence produ...
27       Analyze and understand data to influence produ...
33       Analyze and understand data to influence produ...
46                                                     NaN
                               ...                        
23785                                                  NaN
23786    Analyze and understand data to influence produ...
23798                                                  NaN
23838                                                  NaN
23849    Analyze and understand data to influence produ...
Name: Q11_Part_1, Length: 1922, dtype: object

In [14]:
da_2018['Q11_Part_2']

4                                                      NaN
14                                                     NaN
27                                                     NaN
33       Build and/or run a machine learning service th...
46       Build and/or run a machine learning service th...
                               ...                        
23785                                                  NaN
23786                                                  NaN
23798                                                  NaN
23838                                                  NaN
23849                                                  NaN
Name: Q11_Part_2, Length: 1922, dtype: object

In [15]:
# da_2018['Q11_Part_3']
# ...
da_2018['Q11_OTHER_TEXT']

4       -1
14      -1
27      -1
33      -1
46      -1
        ..
23785   -1
23786   -1
23798   -1
23838   -1
23849   -1
Name: Q11_OTHER_TEXT, Length: 1922, dtype: int64

In [16]:
channel_names = [] # 複選題的選項
channel_checks = [] # 複選題的勾選數
for col_name in da_2018.columns:
        # 將符合複選題的題目特徵（或題號特徵）的變數欄位納入
        # 排除題目特徵中有 OTHER_TEXT 這種開放式回答的變數欄位
        if ('Q11' in col_name) and ('OTHER_TEXT' not in col_name):
            channel_name = da_2018[col_name].value_counts().index # 取出複選題選項
            channel_check = da_2018[col_name].value_counts().values # 取出複選題的勾選數
            # 排除空字串的複選題選項
            if channel_name.size != 0:
                channel_names.append(channel_name[0]) # 存入複選題選項
                channel_checks.append(channel_check[0]) # 存入複選題勾選數
print(channel_names)
print(channel_checks)

['Analyze and understand data to influence product or business decisions', 'Build and/or run a machine learning service that operationally improves my product or workflows', 'Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data', 'Build prototypes to explore applying machine learning to new areas', 'Do research that advances the state of the art of machine learning', 'None of these activities are an important part of my role at work', 'Other']
[1287, 360, 610, 426, 253, 97, 69]


In [17]:
channel_counts = pd.Series(channel_checks, index=channel_names) # 製作一個好排序的 pandas Series
channel_counts

Analyze and understand data to influence product or business decisions                                              1287
Build and/or run a machine learning service that operationally improves my product or workflows                      360
Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data     610
Build prototypes to explore applying machine learning to new areas                                                   426
Do research that advances the state of the art of machine learning                                                   253
None of these activities are an important part of my role at work                                                     97
Other                                                                                                                 69
dtype: int64

In [18]:
channel_ratios = channel_counts / channel_counts.sum() # 加總
channel_ratios

Analyze and understand data to influence product or business decisions                                              0.414894
Build and/or run a machine learning service that operationally improves my product or workflows                     0.116054
Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data    0.196647
Build prototypes to explore applying machine learning to new areas                                                  0.137331
Do research that advances the state of the art of machine learning                                                  0.081560
None of these activities are an important part of my role at work                                                   0.031270
Other                                                                                                               0.022244
dtype: float64

In [19]:
channel_ratios_sorted = channel_ratios.sort_values(ascending=False) # 遞減（由大到小）排序複選題的勾選數
channel_ratios_sorted

Analyze and understand data to influence product or business decisions                                              0.414894
Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data    0.196647
Build prototypes to explore applying machine learning to new areas                                                  0.137331
Build and/or run a machine learning service that operationally improves my product or workflows                     0.116054
Do research that advances the state of the art of machine learning                                                  0.081560
None of these activities are an important part of my role at work                                                   0.031270
Other                                                                                                               0.022244
dtype: float64

In [20]:
# 不想重複做前述的事情，直接呼叫 get_checkbox_ratios 函式
get_checkbox_ratios(da_2018, 'Q11')

Analyze and understand data to influence product or business decisions                                              0.414894
Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data    0.196647
Build prototypes to explore applying machine learning to new areas                                                  0.137331
Build and/or run a machine learning service that operationally improves my product or workflows                     0.116054
Do research that advances the state of the art of machine learning                                                  0.081560
None of these activities are an important part of my role at work                                                   0.031270
Other                                                                                                               0.022244
dtype: float64

此部分的自訂函數較為複雜，對沒有涉獵 Python 程式設計以及 pandas 套件的同學而言，可能還是不太能理解若沒辦法完全理解，沒有關係的喔！只需知道此函數是為了處理複選題中，每個選項被分成不同變數欄位回答，透過 `get_checkbox_ratios`，可將不同選項彙整在一起並計算出每個選項的比例。